In [174]:
import pandas as pd
import numpy as np
import datetime
import math
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [175]:
# list of team names currently in associated leagues (including relegated teams from last season to fill DF), as they are displayed on Understat.

epl_rank = ['Manchester City', 'Arsenal', 'Liverpool', 'Manchester United', 'Newcastle United', 'Tottenham', 'Brighton',
            'Aston Villa', 'Brentford', 'Chelsea', 'Fulham', 'Crystal Palace', 'Wolverhampton Wanderers', 'Everton',
            'West Ham', 'Nottingham Forest', 'Bournemouth', 'Leicester', 'Southampton','Burnley','Luton','Sheffield United',
            'Leeds']
ligue1_rank = ['Ajaccio', 'Marseille', 'Auxerre', 'Lens', 'Monaco', 'Toulouse', 'Nantes', 'Angers', 'Lorient',
               'Strasbourg', 'Nice', 'Lyon', 'Paris Saint Germain', 'Clermont Foot', 'Brest', 'Rennes', 'Reims',
               'Montpellier', 'Troyes', 'Lille','Metz','Le Havre']
bundesliga_rank = ['Borussia Dortmund', 'Mainz 05', 'RasenBallsport Leipzig', 'Schalke 04', 'Union Berlin',
                   'Werder Bremen', 'FC Cologne', 'Bayern Munich', 'Borussia M.Gladbach', 'Augsburg',
                   'Eintracht Frankfurt', 'Freiburg', 'Wolfsburg', 'Hertha Berlin', 'Bochum', 'Bayer Leverkusen',
                   'VfB Stuttgart', 'Hoffenheim','FC Heidenheim','Darmstadt']
serieA_rank = ['Sassuolo', 'Fiorentina', 'Torino', 'Inter', 'Cremonese', 'Salernitana', 'Empoli', 'Lazio', 'Napoli',
               'Sampdoria', 'Atalanta', 'Monza', 'Lecce', 'Bologna', 'AC Milan', 'Verona', 'Roma', 'Spezia', 'Udinese',
               'Juventus','Cagliari','Frosinone','Genoa']
laliga_rank = ['Real Madrid', 'Athletic Club', 'Villarreal', 'Athletico Madrid', 'Celta Vigo', 'Barcelona',
               'Real Valladolid', 'Getafe', 'Elche', 'Cadiz', 'Espanyol', 'Almeria', 'Osasuna', 'Girona', 'Real Betis',
               'Valencia', 'Mallorca', 'Rayo Vallecano', 'Real Sociedad', 'Sevilla','Granada','Alaves','Las Palmas']

In [176]:
fixtures = pd.read_csv(r'fixtures.txt')
# fixtures = fixtures.reset_index()
fixtures
TeamA = []
TeamB = []
for item in fixtures["-- Date"]:
    if "--" not in item:
        items = item.split(" v ")
        TeamA.append(items[0])
        TeamB.append(items[1])
    else:
        continue

In [178]:
# Machine Learning Approach (Logistic Regression) to predict winning teams
def win_ml(df):

    y_win = df.win
    x_win = df.drop(['win','draw','lose','G','GC','Pts'], axis = 1)
    # xtrain_win, xtest_win, ytrain_win, ytest_win = train_test_split(x_win,y_win, train_size = 0.8, random_state = 42)
    # model_win = LogisticRegression()
    # model_win.fit(xtrain_win,ytrain_win)
    # y_preds_win = model_win.predict(xtest_win)
    # accuracy_win = accuracy_score(ytest_win, y_preds_win)
    # return accuracy_win, model_win
    return x_win
eng_df = pd.read_excel(r'../EDA_DF/EPL_understat.xlsx',sheet_name='gamesheet')

x = win_ml(eng_df)
x

# my_params = np.array([[2,1.05,20,10,2,1,1,0,0,1,0,3,1]])
# model_win.predict(my_params)

,Unnamed: 0,date,ID,Team,H/A,xG,xGC,S,SC,QS,...,CS,first_goal,first_conc,played,date_int,match_ID,Rank,0_0,btts_if_g,btts_if_c
0,0,2024-03-04 20:00:00,Sheffield United22163,Sheffield United,h,0.201,2.485,5,22,0,...,0,0,1,1,3.898176,22163,1,0,0,0
1,1,2024-03-04 20:00:00,Arsenal22163,Arsenal,a,2.485,0.201,22,5,1,...,1,1,0,1,3.898176,22163,3,0,0,0
2,2,2024-03-03 15:30:00,Manchester City22160,Manchester City,h,2.643,0.154,27,2,3,...,0,0,1,1,5.085676,22160,1,0,0,1
3,3,2024-03-03 15:30:00,Manchester United22160,Manchester United,a,0.154,2.643,2,27,0,...,0,1,0,1,5.085676,22160,1,0,1,0
4,4,2024-03-03 13:00:00,Bournemouth22156,Bournemouth,a,1.436,1.619,10,20,2,...,1,1,0,1,5.189843,22156,3,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,755,2023-03-17 20:00:00,Nottingham Forest18479,Nottingham Forest,h,0.533,2.500,5,15,1,...,0,1,0,1,356.898176,18479,2,0,1,0
756,756,2023-03-12 14:00:00,West Ham18471,West Ham,h,2.146,2.046,17,12,2,...,0,0,1,1,362.148176,18471,1,0,0,1
757,757,2023-03-12 14:00:00,Aston Villa18471,Aston Villa,a,2.046,2.146,12,17,3,...,0,1,0,1,362.148176,18471,2,0,1,0
758,758,2023-03-11 15:00:00,Tottenham18470,Tottenham,h,1.735,1.628,15,9,1,...,0,1,0,1,363.106510,18470,3,0,1,0


In [171]:
def score_gen(h,a):
    # First Goal
    h_fg = (h['first_goal'].item() + a['first_conc'].item())/2
    # print(f'hfg is {h_fg}')
    a_fg = (a['first_goal'].item() + h['first_conc'].item())/2
    # print(f'afg is {a_fg}')

    a_btts_if_fgh = (h['btts_if_g'].item() + a['btts_if_c'].item())/2
    # print(f'abttsiffgh is {h_fg}')

    h_btts_if_fga = (a['btts_if_g'].item() + h['btts_if_c'].item())/2
    # print(f'hbttsiffga is {h_fg}')

    if h_fg > 0.65:
        if a_btts_if_fgh < 0.5:
            # print("WTN!")
            h_s = round(((h['GD'].item() - a['GD'].item())/2),2)
            a_s = 0
        else:
            h_s = ((h['G'].item() + a['GC'].item())/2)
            a_s = ((a['G'].item() + h['GC'].item())/2)
    elif a_fg > 0.65:
        if h_btts_if_fga < 0.5:
            # print("WTN!")
            a_s = round(((a['GD'].item() - h['GD'].item())/2),0)
            h_s = 0
        else:
            h_s = ((h['G'].item() + a['GC'].item())/2)
            a_s = ((a['G'].item() + h['GC'].item())/2)
    else:
        h_s = ((h['G'].item() + a['GC'].item())/2)
        a_s = ((a['G'].item() + h['GC'].item())/2)

    return h_s, a_s, h['G'].item(), a['G'].item(), h['xG'].item(), a['xG'].item(), h['GD'].item(),a['GD'].item(), h_fg, a_fg, a_btts_if_fgh, h_btts_if_fga # score_h, score_a, GH, GA, xGH, xGA, FGH, FGA, BTTSIFHG, BTTSIFAG

In [172]:

def rank_decision():
    scoresheet_df = pd.DataFrame(columns = ['score_list','score_h','score_a','GH','GA','xGH','xGA','GDH','GDA','FGH','FGA','BTTSIFHG','BTTSIFAG'])
    scoresheet_dict = {}

    for i in range(0,len(TeamA)):
        Home_Team = TeamA[i]
        Away_Team = TeamB[i]

        if Home_Team in epl_rank:
            df = pd.read_excel(r'../EDA_DF/EPL_understat.xlsx',sheet_name='gamesheet')
            l10_table_h = pd.read_excel(r'../EDA_DF/EPL_understat.xlsx',sheet_name='last10_home')
            l10_table_a = pd.read_excel(r'../EDA_DF/EPL_understat.xlsx',sheet_name='last10_away')
            l10_df = pd.read_excel(r'../EDA_DF/EPL_understat.xlsx',sheet_name='last10_gamesheet')
            table_h = pd.read_excel(r'../EDA_DF/EPL_understat.xlsx',sheet_name='home')
            table_a = pd.read_excel(r'../EDA_DF/EPL_understat.xlsx',sheet_name='away')
            r1_table_h = pd.read_excel(r'../EDA_DF/EPL_understat.xlsx',sheet_name='rank1_home')
            r2_table_h = pd.read_excel(r'../EDA_DF/EPL_understat.xlsx',sheet_name='rank2_home')
            r3_table_h = pd.read_excel(r'../EDA_DF/EPL_understat.xlsx',sheet_name='rank3_home')
            r1_table_a = pd.read_excel(r'../EDA_DF/EPL_understat.xlsx',sheet_name='rank1_away')
            r2_table_a = pd.read_excel(r'../EDA_DF/EPL_understat.xlsx',sheet_name='rank2_away')
            r3_table_a = pd.read_excel(r'../EDA_DF/EPL_understat.xlsx',sheet_name='rank3_away')

        elif Home_Team in ligue1_rank:
            df = pd.read_excel(r'../EDA_DF/Ligue1_understat.xlsx', sheet_name='gamesheet')
            l10_table_h = pd.read_excel(r'../EDA_DF/Ligue1_understat.xlsx',sheet_name='last10_home')
            l10_table_a = pd.read_excel(r'../EDA_DF/Ligue1_understat.xlsx',sheet_name='last10_away')
            l10_df = pd.read_excel(r'../EDA_DF/Ligue1_understat.xlsx',sheet_name='last10_gamesheet')
            table_h = pd.read_excel(r'../EDA_DF/Ligue1_understat.xlsx',sheet_name='home')
            table_a = pd.read_excel(r'../EDA_DF/Ligue1_understat.xlsx',sheet_name='away')
            r1_table_h = pd.read_excel(r'../EDA_DF/Ligue1_understat.xlsx',sheet_name='rank1_home')
            r2_table_h = pd.read_excel(r'../EDA_DF/Ligue1_understat.xlsx',sheet_name='rank2_home')
            r3_table_h = pd.read_excel(r'../EDA_DF/Ligue1_understat.xlsx',sheet_name='rank3_home')
            r1_table_a = pd.read_excel(r'../EDA_DF/Ligue1_understat.xlsx',sheet_name='rank1_away')
            r2_table_a = pd.read_excel(r'../EDA_DF/Ligue1_understat.xlsx',sheet_name='rank2_away')
            r3_table_a = pd.read_excel(r'../EDA_DF/Ligue1_understat.xlsx',sheet_name='rank3_away')

        elif Home_Team in bundesliga_rank:
            df = pd.read_excel(r'../EDA_DF/Bundesliga_understat.xlsx', sheet_name='gamesheet')
            l10_table_h = pd.read_excel(r'../EDA_DF/Bundesliga_understat.xlsx',sheet_name='last10_home')
            l10_table_a = pd.read_excel(r'../EDA_DF/Bundesliga_understat.xlsx',sheet_name='last10_away')
            l10_df = pd.read_excel(r'../EDA_DF/Bundesliga_understat.xlsx',sheet_name='last10_gamesheet')
            table_h = pd.read_excel(r'../EDA_DF/Bundesliga_understat.xlsx',sheet_name='home')
            table_a = pd.read_excel(r'../EDA_DF/Bundesliga_understat.xlsx',sheet_name='away')
            r1_table_h = pd.read_excel(r'../EDA_DF/Bundesliga_understat.xlsx',sheet_name='rank1_home')
            r2_table_h = pd.read_excel(r'../EDA_DF/Bundesliga_understat.xlsx',sheet_name='rank2_home')
            r3_table_h = pd.read_excel(r'../EDA_DF/Bundesliga_understat.xlsx',sheet_name='rank3_home')
            r1_table_a = pd.read_excel(r'../EDA_DF/Bundesliga_understat.xlsx',sheet_name='rank1_away')
            r2_table_a = pd.read_excel(r'../EDA_DF/Bundesliga_understat.xlsx',sheet_name='rank2_away')
            r3_table_a = pd.read_excel(r'../EDA_DF/Bundesliga_understat.xlsx',sheet_name='rank3_away')

        elif Home_Team in serieA_rank:
            df = pd.read_excel(r'../EDA_DF/SerieA_understat.xlsx', sheet_name='gamesheet')
            l10_table_h = pd.read_excel(r'../EDA_DF/SerieA_understat.xlsx',sheet_name='last10_home')
            l10_table_a = pd.read_excel(r'../EDA_DF/SerieA_understat.xlsx',sheet_name='last10_away')
            l10_df = pd.read_excel(r'../EDA_DF/SerieA_understat.xlsx',sheet_name='last10_gamesheet')
            table_h = pd.read_excel(r'../EDA_DF/SerieA_understat.xlsx',sheet_name='home')
            table_a = pd.read_excel(r'../EDA_DF/SerieA_understat.xlsx',sheet_name='away')
            r1_table_h = pd.read_excel(r'../EDA_DF/SerieA_understat.xlsx',sheet_name='rank1_home')
            r2_table_h = pd.read_excel(r'../EDA_DF/SerieA_understat.xlsx',sheet_name='rank2_home')
            r3_table_h = pd.read_excel(r'../EDA_DF/SerieA_understat.xlsx',sheet_name='rank3_home')
            r1_table_a = pd.read_excel(r'../EDA_DF/SerieA_understat.xlsx',sheet_name='rank1_away')
            r2_table_a = pd.read_excel(r'../EDA_DF/SerieA_understat.xlsx',sheet_name='rank2_away')
            r3_table_a = pd.read_excel(r'../EDA_DF/SerieA_understat.xlsx',sheet_name='rank3_away')

        elif Home_Team in laliga_rank:
            df = pd.read_excel(r'../EDA_DF/LaLiga_understat.xlsx', sheet_name='gamesheet')
            l10_table_h = pd.read_excel(r'../EDA_DF/LaLiga_understat.xlsx',sheet_name='last10_home')
            l10_table_a = pd.read_excel(r'../EDA_DF/LaLiga_understat.xlsx',sheet_name='last10_away')
            l10_df = pd.read_excel(r'../EDA_DF/LaLiga_understat.xlsx',sheet_name='last10_gamesheet')
            table_h = pd.read_excel(r'../EDA_DF/LaLiga_understat.xlsx',sheet_name='home')
            table_a = pd.read_excel(r'../EDA_DF/LaLiga_understat.xlsx',sheet_name='away')
            r1_table_h = pd.read_excel(r'../EDA_DF/LaLiga_understat.xlsx',sheet_name='rank1_home')
            r2_table_h = pd.read_excel(r'../EDA_DF/LaLiga_understat.xlsx',sheet_name='rank2_home')
            r3_table_h = pd.read_excel(r'../EDA_DF/LaLiga_understat.xlsx',sheet_name='rank3_home')
            r1_table_a = pd.read_excel(r'../EDA_DF/LaLiga_understat.xlsx',sheet_name='rank1_away')
            r2_table_a = pd.read_excel(r'../EDA_DF/LaLiga_understat.xlsx',sheet_name='rank2_away')
            r3_table_a = pd.read_excel(r'../EDA_DF/LaLiga_understat.xlsx',sheet_name='rank3_away')
            
        else:
            print("This Team doesnt feature in our database!")
            i += 1
        
        # Historic Rank data vs Form-Influenced Rank Data
        hf = l10_table_h.loc[l10_table_h["Team"] == Home_Team]
        hrec = table_h.loc[table_h["Team"] == Home_Team]
        af = l10_table_a.loc[l10_table_a["Team"] == Away_Team]
        arec = table_a.loc[table_a["Team"] == Away_Team]
        hf_rank = hf['Rank'].item()
        hf_rankvs = hf['Rankvs'].item()
        hrec = hrec['Rank'].item()
        af_rank = af['Rank'].item()
        af_rankvs = af['Rankvs'].item()
        arec = arec['Rank'].item()
        # Home
        if (hf_rank == hrec) or (hf_rank < hrec and hf_rankvs > 2.25) or (hf_rank > hrec and hf_rankvs < 1.75):
            # print("same rank OR rankvs significant")
            if hf_rank == 1:
                away = r1_table_a.loc[r1_table_a["Team"] == Away_Team]
            elif hf_rank == 2:
                away = r2_table_a.loc[r2_table_a["Team"] == Away_Team]
            else:
                away = r3_table_a.loc[r3_table_a["Team"] == Away_Team]
        elif (hf_rank < hrec and hf_rankvs <= 2.25):
            # print("Better form")
            if hf_rank == 1:
                away = r1_table_a.loc[r1_table_a["Team"] == Away_Team]
            else:
                away = r2_table_a.loc[r2_table_a["Team"] == Away_Team]
        elif (hf_rank > hrec and hf_rankvs >= 1.75):
            # print("Worse Form")
            if hf_rank == 2:
                away = r2_table_a.loc[r2_table_a["Team"] == Away_Team]
            else:
                away = r3_table_a.loc[r3_table_a["Team"] == Away_Team]
        else:
            print("???")
        # Away
        if (af_rank == arec) or (af_rank < arec and af_rankvs > 2.25) or (af_rank > arec and af_rankvs < 1.75):
            # print("same rank OR rankvs significant")
            if af_rank == 1:
                home = r1_table_h.loc[r1_table_h["Team"] == Home_Team]
            elif af_rank == 2:
                home = r2_table_h.loc[r2_table_h["Team"] == Home_Team]
            else:
                home = r3_table_h.loc[r3_table_h["Team"] == Home_Team]
        elif (af_rank < arec and af_rankvs <= 2.25):
            # print("Better form")
            if af_rank == 1:
                home = r1_table_h.loc[r1_table_h["Team"] == Home_Team]
            else:
                home = r2_table_h.loc[r2_table_h["Team"] == Home_Team]
        elif (af_rank > arec and af_rankvs >= 1.75):
            # print("Worse Form")
            if af_rank == 2:
                home = r2_table_h.loc[r2_table_h["Team"] == Home_Team]
            else:
                home = r3_table_h.loc[r3_table_h["Team"] == Home_Team]
        else:
            print("???")
        score_h, score_a, GH, GA, xGH, xGA, GDH, GDA, FGH, FGA, BTTSIFHG, BTTSIFAG = score_gen(home,away)
        # print(f"{home['Team'].item()} v {away['Team'].item()}")
        score_list = (f"{home['Team'].item()} {score_h} - {score_a} {away['Team'].item()}")
        # scoresheet_list.append(score_list)
        col_names = ['score_list','score_h','score_a','GH','GA','xGH','xGA','GDH','GDA','FGH','FGA','BTTSIFHG','BTTSIFAG']
        df_game = pd.DataFrame([score_list,score_h,score_a,GH,GA,xGH,xGA,GDH,GDA,FGH,FGA,BTTSIFHG,BTTSIFAG],index=col_names)
        df_game = df_game.T
        scoresheet_df = pd.concat([scoresheet_df, df_game])
        i += 1
    return scoresheet_df
x = rank_decision()
x

,score_list,score_h,score_a,GH,GA,xGH,xGA,GDH,GDA,FGH,FGA,BTTSIFHG,BTTSIFAG
0,Manchester United 1.4166666666666665 - 1.375 E...,1.416667,1.375,1.75,1.5,2.63075,1.572083,0.5,0.416667,0.291667,0.625,0.375,0.833333
0,Bournemouth 2.7 - 1.15 Sheffield United,2.7,1.15,2.4,1.5,2.0826,1.3005,1.6,-1.5,0.55,0.45,0.333333,1.0
0,Crystal Palace 2.2666666666666666 - 1.51666666...,2.266667,1.516667,1.7,1.333333,1.4166,1.372833,0.0,-1.5,0.7,0.25,0.708333,0.4
0,Wolverhampton Wanderers 0.85 - 0 Fulham,0.85,0,1.6,0.2,1.4975,0.6048,0.5,-1.2,0.7,0.2,0.291667,0.375
0,Arsenal 1.6666666666666665 - 0 Brentford,1.666667,0,3.166667,0.25,2.5795,0.79075,1.833333,-1.5,0.708333,0.291667,0.375,1.0
0,Aston Villa 2.8055555555555554 - 1.27777777777...,2.805556,1.277778,3.111111,1.666667,2.347333,1.245167,2.222222,-0.833333,0.555556,0.444444,0.785714,0.875
0,Brighton 2.227272727272727 - 1.454545454545454...,2.227273,1.454545,2.545455,1.272727,2.408364,1.332818,0.909091,-0.636364,0.363636,0.545455,0.75,0.916667
0,West Ham 1.25 - 0 Burnley,1.25,0,2.25,0.571429,2.3585,0.689714,1.5,-1.0,0.660714,0.339286,0.166667,0.833333
0,Liverpool 1.0333333333333332 - 0.9 Manchester ...,1.033333,0.9,1.4,1.0,2.2694,2.135333,0.6,0.333333,0.433333,0.466667,0.0,0.5
0,Chelsea 1.9875 - 1.6 Newcastle United,1.9875,1.6,1.375,1.2,1.759875,1.031,-0.625,-1.4,0.7125,0.2875,0.75,0.666667


In [ ]:
def poisson_dist():
    

In [ ]:
with pd.ExcelWriter("../Frontend/scorepredictions.xlsx") as writer:
    x.to_excel(writer, sheet_name="gamesheet")